# Recommender Code Along

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works.

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec').getOrCreate()

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [5]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [6]:
data = spark.read.csv('/FileStore/tables/movielens_ratings.csv',inferSchema=True,header=True)
data.printSchema()

root
-- movieId: integer (nullable = true)
-- rating: double (nullable = true)
-- userId: integer (nullable = true)

In [7]:
data.show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 2| 3.0| 0|
 3| 1.0| 0|
 5| 2.0| 0|
 9| 4.0| 0|
 11| 1.0| 0|
 12| 2.0| 0|
 15| 1.0| 0|
 17| 1.0| 0|
 19| 1.0| 0|
 21| 1.0| 0|
 23| 1.0| 0|
 26| 3.0| 0|
 27| 1.0| 0|
 28| 1.0| 0|
 29| 1.0| 0|
 30| 1.0| 0|
 31| 1.0| 0|
 34| 1.0| 0|
 37| 1.0| 0|
 41| 2.0| 0|
+-------+------+------+
only showing top 20 rows

In [8]:
# Smaller dataset so we will use 0.8 / 0.2
training, test = data.randomSplit([0.8, 0.2])

In [9]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

In [10]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
predictions.show()

+-------+------+------+------------+
movieId|rating|userId| prediction|
+-------+------+------+------------+
 31| 1.0| 27| 1.5933173|
 31| 3.0| 7| 1.3824091|
 31| 2.0| 25| 1.5706278|
 31| 1.0| 24| 0.8164117|
 31| 1.0| 29| 1.9865663|
 85| 1.0| 12| -0.9330596|
 85| 1.0| 5| 2.239271|
 65| 1.0| 28| -0.04961375|
 65| 1.0| 22| -0.09738952|
 65| 2.0| 3| 1.7084935|
 53| 5.0| 21| 1.0545993|
 78| 1.0| 19| -0.27382016|
 78| 1.0| 4| 0.932465|
 78| 1.0| 11|-0.016844422|
 34| 1.0| 28| -1.3082801|
 34| 1.0| 4| 2.6864402|
 81| 1.0| 16| -0.23456047|
 81| 1.0| 21| 0.56601954|
 28| 1.0| 6| 1.4559411|
 28| 1.0| 2| 5.0618343|
+-------+------+------+------------+
only showing top 20 rows

In [11]:
# Use RMSE to describe our error in terms of the stars rating column.

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.7806194875821035

In [12]:
# Supply the recommendation to a user

single_user = test.filter(test['userId']==11).select(['movieId','userId'])

# User had 10 ratings in the test data set, realistically this should be some sort of hold out set.
single_user.show()

+-------+------+
movieId|userId|
+-------+------+
 23| 11|
 25| 11|
 35| 11|
 45| 11|
 59| 11|
 66| 11|
 75| 11|
 78| 11|
+-------+------+

In [13]:
reccomendations = model.transform(single_user)
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+------------+
movieId|userId| prediction|
+-------+------+------------+
 23| 11| 2.3392787|
 25| 11| 1.6914517|
 75| 11| 1.6648116|
 35| 11| 1.6400058|
 59| 11| 1.4744519|
 45| 11| 1.389849|
 66| 11| 1.0206544|
 78| 11|-0.016844422|
+-------+------+------------+